### Importing libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [1]:
import tensorflow as tf
from tensorflow import keras

ModuleNotFoundError: No module named 'tensorflow'

### Importing datafiles

In [31]:
df = pd.read_csv('data-training.csv')

In [32]:
df.head()

,id,menopaus,agegrp,density,race,Hispanic,bmi,agefirst,nrelbc,brstproc,lastmamm,surgmeno,hrt,invasive,cancer,count
0,RE-TRAIN-0000001,0,1,1,1,0,1,0,0,0,9,9,9,0,0,2
1,RE-TRAIN-0000002,0,1,1,1,0,1,0,0,0,9,9,9,0,0,4
2,RE-TRAIN-0000003,0,1,1,1,0,1,0,1,0,0,9,9,0,0,1
3,RE-TRAIN-0000004,0,1,1,1,0,1,0,1,0,9,9,9,0,0,1
4,RE-TRAIN-0000005,0,1,1,1,0,1,1,0,0,0,9,9,0,0,2


In [4]:
data_dic = pd.read_excel('Data Dictionary.xlsx')

In [5]:
pd.set_option('display.max_colwidth', None)  

In [33]:
data_dic

,Column,Column Name,Column Data
0,0.0,id,Identifier of the row
1,1.0,menopaus,0 = premenopausal; 1 = postmenopausal or age>=55 ; 9 = unknown
2,2.0,agegrp,1 = 35-39; 2 = 40-44; 3 = 45-49; 4 = 50-54; 5 = 55-59; 6 = 60-64; 7 = 65-69; 8 = 70-74; 9 = 75-79; 10 = 80-84
3,3.0,density,BI-RADS breast density codes 1 = Almost entirely fat; 2 = Scattered fibroglandular densities; 3 = Heterogeneously dense; 4 = Extremely dense; 9 = Unknown or different measurement system
4,4.0,race,1 = white; 2 = Asian/Pacific Islander; 3 = black; 4 = Native American; 5 = other/mixed; 9 = unknown
5,5.0,Hispanic,0 = no; 1 = yes; 9 = unknown
6,6.0,bmi,Body mass index: 1 = 10-24.99; 2 = 25-29.99; 3 = 30-34.99; 4 = 35 or more; 9 = unknown
7,7.0,agefirst,Age at first birth: 0 = Age < 30; 1 = Age 30 or greater; 2 = Nulliparous; 9 = unknown
8,8.0,nrelbc,Number of first degree relatives with breast cancer: 0 = zero; 1= one; 2 = 2 or more; 9 = unknown
9,9.0,brstproc,Previous breast procedure: 0 = no; 1 = yes; 9 = unknown


### Exploring dataset

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196462 entries, 0 to 196461
Data columns (total 16 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        196462 non-null  object
 1   menopaus  196462 non-null  int64 
 2   agegrp    196462 non-null  int64 
 3   density   196462 non-null  int64 
 4   race      196462 non-null  int64 
 5   Hispanic  196462 non-null  int64 
 6   bmi       196462 non-null  int64 
 7   agefirst  196462 non-null  int64 
 8   nrelbc    196462 non-null  int64 
 9   brstproc  196462 non-null  int64 
 10  lastmamm  196462 non-null  int64 
 11  surgmeno  196462 non-null  int64 
 12  hrt       196462 non-null  int64 
 13  invasive  196462 non-null  int64 
 14  cancer    196462 non-null  int64 
 15  count     196462 non-null  int64 
dtypes: int64(15), object(1)
memory usage: 24.0+ MB


In [35]:
df.columns[1:]

Index(['menopaus', 'agegrp', 'density', 'race', 'Hispanic', 'bmi', 'agefirst',
       'nrelbc', 'brstproc', 'lastmamm', 'surgmeno', 'hrt', 'invasive',
       'cancer', 'count'],
      dtype='object')

In [36]:
corr_mat = df.corr()

In [37]:
corr_mat['cancer'].sort_values(ascending=True)

Hispanic   -0.040359
race       -0.037971
lastmamm   -0.026021
count      -0.024619
brstproc   -0.017272
nrelbc     -0.009726
hrt        -0.005826
menopaus   -0.005108
surgmeno    0.002086
agegrp      0.026089
density     0.026287
bmi         0.037805
agefirst    0.058492
invasive    0.874999
cancer      1.000000
Name: cancer, dtype: float64

In [40]:
from random import sample

In [23]:
def fill_func(col):
    return sample(list(df[df[col]!=9][col].values),len(df[df[col]==9][col]))    

In [74]:
dfnew = df.replace({9: np.nan})

In [39]:
dfnew.head(10)

,id,menopaus,agegrp,density,race,Hispanic,bmi,agefirst,nrelbc,brstproc,lastmamm,surgmeno,hrt,invasive,cancer,count
0,RE-TRAIN-0000001,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,2.0
1,RE-TRAIN-0000002,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,4.0
2,RE-TRAIN-0000003,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,NaN,NaN,0.0,0.0,1.0
3,RE-TRAIN-0000004,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,NaN,NaN,NaN,0.0,0.0,1.0
4,RE-TRAIN-0000005,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,2.0
5,RE-TRAIN-0000006,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,NaN,NaN,0.0,0.0,1.0
6,RE-TRAIN-0000007,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,NaN,NaN,NaN,0.0,0.0,1.0
7,RE-TRAIN-0000008,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,NaN,NaN,0.0,0.0,2.0
8,RE-TRAIN-0000009,0.0,1.0,1.0,1.0,0.0,1.0,1.0,2.0,0.0,0.0,NaN,NaN,0.0,0.0,1.0
9,RE-TRAIN-0000010,0.0,1.0,1.0,1.0,0.0,1.0,2.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,1.0


In [75]:
dfnew['agegrp'] = dfnew['agegrp'].fillna(value=9)

In [76]:
dfnew.loc[dfnew.menopaus == 0.0, 'surgmeno'] = 0.0

In [77]:
dfnew.loc[dfnew.menopaus == 0.0, 'hrt'] = 0.0

In [80]:
dfnew.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196462 entries, 0 to 196461
Data columns (total 16 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   id        196462 non-null  object 
 1   menopaus  185419 non-null  float64
 2   agegrp    196462 non-null  float64
 3   density   146732 non-null  float64
 4   race      156011 non-null  float64
 5   Hispanic  136563 non-null  float64
 6   bmi       119957 non-null  float64
 7   agefirst  132401 non-null  float64
 8   nrelbc    168676 non-null  float64
 9   brstproc  171085 non-null  float64
 10  lastmamm  131401 non-null  float64
 11  surgmeno  139288 non-null  float64
 12  hrt       158414 non-null  float64
 13  invasive  196462 non-null  float64
 14  cancer    196462 non-null  float64
 15  count     194330 non-null  float64
dtypes: float64(15), object(1)
memory usage: 24.0+ MB


In [79]:
fill = pd.DataFrame(index =dfnew.index[dfnew.menopaus.isnull()], data= fill_val,columns=['menopaus'])

NameError: name 'fill_val' is not defined

In [26]:
fill.head()

,menopaus
185419,1
185420,1
185421,1
185422,1
185423,1


In [30]:
df['agegrp'].loc[185419]

1

In [32]:
dfnew.fillna(fill,inplace = True)

In [81]:
def fill_func(col):
    listval = sample(list(dfnew[dfnew[col].isna()==False][col].values),len(dfnew[dfnew[col].isna()][col])) 
    fill = pd.DataFrame(index =dfnew.index[dfnew[col].isnull()], data= listval,columns=[col])
    dfnew.fillna(fill,inplace = True)
    dfnew[col] = dfnew[col].astype(float)

In [72]:
type(dfnew['density'][1])

float

In [82]:
cols = np.array(dfnew.columns)

In [84]:
cols.pop(1)

AttributeError: 'numpy.ndarray' object has no attribute 'pop'

In [85]:
for col in cols[1:]:
    fill_func(col)

In [86]:
dfnew.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196462 entries, 0 to 196461
Data columns (total 16 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   id        196462 non-null  object 
 1   menopaus  196462 non-null  int64  
 2   agegrp    196462 non-null  float64
 3   density   196462 non-null  float64
 4   race      196462 non-null  float64
 5   Hispanic  196462 non-null  float64
 6   bmi       196462 non-null  float64
 7   agefirst  196462 non-null  float64
 8   nrelbc    196462 non-null  float64
 9   brstproc  196462 non-null  float64
 10  lastmamm  196462 non-null  float64
 11  surgmeno  196462 non-null  float64
 12  hrt       196462 non-null  float64
 13  invasive  196462 non-null  float64
 14  cancer    196462 non-null  float64
 15  count     196462 non-null  float64
dtypes: float64(14), int64(1), object(1)
memory usage: 24.0+ MB


In [88]:
dfnew.corr()['cancer'].sort_values(ascending=True)

race       -0.050900
Hispanic   -0.049136
nrelbc     -0.026613
lastmamm   -0.024811
count      -0.024415
bmi        -0.023146
brstproc   -0.013001
surgmeno   -0.012786
agefirst   -0.010378
menopaus   -0.000015
hrt         0.006973
density     0.022541
agegrp      0.026089
invasive    0.874999
cancer      1.000000
Name: cancer, dtype: float64

In [25]:
df.corr()['cancer'].sort_values(ascending=True)

Hispanic   -0.040359
race       -0.037971
lastmamm   -0.026021
count      -0.024619
brstproc   -0.017272
nrelbc     -0.009726
hrt        -0.005826
menopaus   -0.005108
surgmeno    0.002086
agegrp      0.026089
density     0.026287
bmi         0.037805
agefirst    0.058492
invasive    0.874999
cancer      1.000000
Name: cancer, dtype: float64

### Logistic Regression

In [89]:
x_train = dfnew.drop(['id','cancer'],axis=1)
y_train = dfnew['cancer']

In [96]:
x_train.head()

,menopaus,agegrp,density,race,Hispanic,bmi,agefirst,nrelbc,brstproc,lastmamm,surgmeno,hrt,invasive,count
0,0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
1,0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
2,0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0


In [92]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
#lr.fit(x_train, y_train)
#print("score on train: "+ str(lr.score(x_train, y_train)))

In [93]:
from pprint import pprint
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(lr.get_params())

Parameters currently in use:

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}


In [94]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
# Number of trees in random forest
cval = [0.001, 0.01, 0.1, 1.0]
solver = ['lbfgs','saga']
maxiter = [100, 1000]

# Create the random grid
random_grid = {'C': cval,
               'solver': solver,
               'max_iter': maxiter
              }
print(random_grid)

{'C': [0.001, 0.01, 0.1, 1.0], 'solver': ['lbfgs', 'saga'], 'max_iter': [100, 1000]}


In [95]:
grid_search = GridSearchCV(estimator = lr, param_grid = random_grid, 
                          cv = 10, n_jobs = -1, verbose = 2)

In [97]:
grid_search.fit(x_train, y_train)

Fitting 10 folds for each of 16 candidates, totalling 160 fits


GridSearchCV(cv=10, estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': [0.001, 0.01, 0.1, 1.0], 'max_iter': [100, 1000],
                         'solver': ['lbfgs', 'saga']},
             verbose=2)

In [98]:
grid_search.best_params_

{'C': 1.0, 'max_iter': 100, 'solver': 'lbfgs'}

In [72]:
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

In [73]:
y_train_pred = cross_val_predict(lr, x_train, y_train, cv=10) 

In [74]:
print(precision_score(y_train, y_train_pred))
print(recall_score(y_train, y_train_pred))
print(f1_score(y_train, y_train_pred))

1.0
0.7700920245398774
0.8701152413135778


### Preparing test data set

In [99]:
testdf = pd.read_csv('data-provisional.csv')

In [100]:
testdf.head()

,id,menopaus,agegrp,density,race,Hispanic,bmi,agefirst,nrelbc,brstproc,lastmamm,surgmeno,hrt,invasive,count
0,RE-PROV-0000001,0,1,1,1,0,1,0,0,0,0,9,9,0,4
1,RE-PROV-0000002,0,1,1,1,0,2,0,1,0,9,9,9,0,1
2,RE-PROV-0000003,0,1,1,1,0,2,9,0,0,0,9,9,0,4
3,RE-PROV-0000004,0,1,1,1,0,2,9,1,0,0,9,9,0,2
4,RE-PROV-0000005,0,1,1,1,0,3,2,0,0,9,9,9,0,3


In [110]:
testdfnew.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28066 entries, 0 to 28065
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        28066 non-null  object
 1   menopaus  28066 non-null  int64 
 2   agegrp    28066 non-null  int64 
 3   density   28066 non-null  object
 4   race      28066 non-null  object
 5   Hispanic  28066 non-null  object
 6   bmi       28066 non-null  object
 7   agefirst  28066 non-null  object
 8   nrelbc    28066 non-null  object
 9   brstproc  28066 non-null  object
 10  lastmamm  28066 non-null  object
 11  surgmeno  28066 non-null  object
 12  hrt       28066 non-null  object
 13  invasive  28066 non-null  int64 
 14  count     28066 non-null  object
dtypes: int64(3), object(12)
memory usage: 3.2+ MB


In [112]:
testdfnew = testdf.replace({9: np.nan})

In [113]:
testdfnew['agegrp'] = testdfnew['agegrp'].fillna(value=9)

In [114]:
testdfnew.loc[testdfnew.menopaus == 0.0, 'surgmeno'] = 0.0
testdfnew.loc[testdfnew.hrt == 0.0, 'hrt'] = 0.0

In [115]:
def fill_func(col):
    listval = sample(list(testdfnew[testdfnew[col].isna()==False][col].values),len(testdfnew[testdfnew[col].isna()][col])) 
    fill = pd.DataFrame(index =testdfnew.index[testdfnew[col].isnull()], data= listval,columns=[col])
    testdfnew.fillna(fill,inplace = True)
    testdfnew[col] = testdfnew[col].astype(float)

In [107]:
cols = np.array(testdfnew.columns)

In [108]:
cols

array(['id', 'menopaus', 'agegrp', 'density', 'race', 'Hispanic', 'bmi',
       'agefirst', 'nrelbc', 'brstproc', 'lastmamm', 'surgmeno', 'hrt',
       'invasive', 'count'], dtype=object)

In [116]:
for col in cols[1:]:
    fill_func(col)

In [117]:
x_test = testdfnew.drop('id',axis=1)

In [118]:
x_test.head()

,menopaus,agegrp,density,race,Hispanic,bmi,agefirst,nrelbc,brstproc,lastmamm,surgmeno,hrt,invasive,count
0,0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0
1,0,1.0,1.0,1.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
2,0,1.0,1.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
3,0,1.0,1.0,1.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0
4,0,1.0,1.0,1.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0


In [128]:
x_train.head()

,menopaus,agegrp,density,race,Hispanic,bmi,agefirst,nrelbc,brstproc,lastmamm,surgmeno,hrt,invasive,count
0,0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
1,0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
2,0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0


In [91]:
predic = lr.predict(x_test)

In [92]:
pid = testdfnew['id'].array
d = {'id' : pid,'prediction':predic}
savedat = pd.DataFrame(d)
savedat.to_csv('lr.csv',index=False)

### Random Forest

In [126]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state = 42)
from pprint import pprint
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

Parameters currently in use:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


In [129]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 100, num = 5)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 20, num = 4)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
#bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               #'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf #,
               #'bootstrap': bootstrap
              }
print(random_grid)

{'n_estimators': [10, 32, 55, 77, 100], 'max_depth': [5, 10, 15, 20, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]}


In [130]:
grid_search = GridSearchCV(estimator = rf, param_grid = random_grid, 
                          cv = 5, n_jobs = -1, verbose = 2)

In [131]:
grid_search.fit(x_train, y_train)

Fitting 5 folds for each of 225 candidates, totalling 1125 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [5, 10, 15, 20, None],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [10, 32, 55, 77, 100]},
             verbose=2)

In [132]:
grid_search.best_params_

{'max_depth': 5,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 77}

In [133]:
best_rf = RandomForestClassifier(bootstrap = True, max_depth = 5, min_samples_leaf = 1,
                                 min_samples_split = 2, n_estimators = 77, random_state = 42)

In [136]:
y_train_pred = cross_val_predict(best_rf, x_train, y_train, cv=5) 

In [135]:
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

In [140]:
best_rf.fit(x_train, y_train)

RandomForestClassifier(max_depth=5, n_estimators=77, random_state=42)

In [138]:
print(precision_score(y_train, y_train_pred))
print(recall_score(y_train, y_train_pred))
print(f1_score(y_train, y_train_pred))

1.0
0.7716257668711657
0.871093411825816


In [141]:
predic = best_rf.predict(x_test)

In [142]:
pid = testdfnew['id'].array
d = {'id' : pid,'prediction':predic}
savedat = pd.DataFrame(d)
savedat.to_csv('rf.csv',index=False)